In [35]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,  confusion_matrix

In [59]:
df = pd.read_csv("cccna_data.csv")
df

,name,bind,V800,V801,V802,V803,V804,V805,V806,V807,...,V1191,V1192,V1193,V1194,V1195,V1196,V1197,V1198,V1199,V1200
0,peak_10,True,3,2,4,2,4,1,2,2,...,3,4,1,2,1,3,4,1,4,2
1,peak_10003,True,1,4,1,3,2,1,4,2,...,1,2,4,2,1,4,1,4,2,2
2,peak_10004,True,4,1,3,4,1,3,3,1,...,2,1,2,4,1,3,4,4,1,2
3,peak_10006,True,2,1,2,4,1,3,1,1,...,4,1,3,4,2,3,3,1,1,3
4,peak_10011,False,3,3,2,2,2,1,4,2,...,1,3,1,3,2,1,2,2,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51044,peak_9961,False,3,2,2,2,2,3,2,3,...,2,3,3,2,3,1,3,4,1,1
51045,peak_9962,True,1,1,1,4,1,4,1,4,...,2,4,4,2,3,3,2,2,1,2
51046,peak_9985,True,4,1,4,3,3,2,2,2,...,1,3,2,3,2,2,2,3,4,1
51047,peak_9986,False,1,4,1,4,4,2,3,3,...,2,3,1,4,1,2,4,1,4,2


In [114]:
df.bind.value_counts()

bind
True     29018
False    22031
Name: count, dtype: int64

In [60]:
df.isna().any().any()

np.False_

In [61]:
df.duplicated().any()

np.False_

In [121]:
t = df[df["bind"] == True].iloc[:22031]
f = df[df["bind"] == False]
df_eq = pd.merge(t, f, how="outer")
df_eq

,name,bind,V800,V801,V802,V803,V804,V805,V806,V807,...,V1191,V1192,V1193,V1194,V1195,V1196,V1197,V1198,V1199,V1200
0,peak_10,True,3,2,4,2,4,1,2,2,...,3,4,1,2,1,3,4,1,4,2
1,peak_10002,True,2,4,2,4,1,3,2,3,...,1,2,1,3,2,2,1,2,1,1
2,peak_10003,True,1,4,1,3,2,1,4,2,...,1,2,4,2,1,4,1,4,2,2
3,peak_10004,True,4,1,3,4,1,3,3,1,...,2,1,2,4,1,3,4,4,1,2
4,peak_10005,False,1,2,4,1,1,4,2,1,...,4,2,3,3,3,1,1,2,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44057,peak_9986,False,1,4,1,4,4,2,3,3,...,2,3,1,4,1,2,4,1,4,2
44058,peak_9987,True,1,3,4,1,1,1,1,3,...,2,1,2,3,3,1,3,1,4,3
44059,peak_9991,True,3,1,3,1,4,1,4,2,...,2,2,3,1,3,1,2,4,2,4
44060,peak_9992,True,2,1,3,3,2,4,4,1,...,1,1,1,4,1,1,1,4,1,1


In [62]:
ss = StandardScaler()

In [122]:
x = df_eq.drop(["name", "bind"], axis=1)
y = df_eq[["bind"]]

In [123]:
x_standart = ss.fit_transform(x)

In [124]:
x_train, x_test, y_train, y_test = train_test_split(x_standart, y, test_size=0.3, random_state=42)

# Логистическая регрессия

In [125]:
lr = LogisticRegression()

In [126]:
lr.fit(x_train, y_train)

/home/alwalone/anaconda3/lib/python3.12/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [127]:
y_pred = lr.predict(x_test)

In [128]:
print(accuracy_score(y_test, y_pred))
print(precision_score(y_test, y_pred))
print(recall_score(y_test, y_pred))
print(f1_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

0.6671457750208034
0.6623741859088218
0.6786472550803762
0.6704119850187266
[[4344 2281]
 [2119 4475]]


# Дерево решений

In [129]:
param_grid = {
    "criterion": ['gini', 'entropy', 'log_loss'],
    "max_depth": list(range(4, 100)),
    "max_features": ['sqrt', 'log2'],
    "min_samples_leaf": list(range(1, 10)),
    "min_samples_split": list(range(2, 11)),
    "random_state": list(range(30, 60))
}

In [130]:
dtc = DecisionTreeClassifier()

In [131]:
rs = RandomizedSearchCV(
    estimator=dtc,
    param_distributions=param_grid,
    n_iter=50,
    scoring="accuracy",
    verbose=1,
    n_jobs=-1
)

In [132]:
rs.fit(x_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


RandomizedSearchCV(estimator=DecisionTreeClassifier(), n_iter=50, n_jobs=-1,
                   param_distributions={'criterion': ['gini', 'entropy',
                                                      'log_loss'],
                                        'max_depth': [4, 5, 6, 7, 8, 9, 10, 11,
                                                      12, 13, 14, 15, 16, 17,
                                                      18, 19, 20, 21, 22, 23,
                                                      24, 25, 26, 27, 28, 29,
                                                      30, 31, 32, 33, ...],
                                        'max_features': ['sqrt', 'log2'],
                                        'min_samples_leaf': [1, 2, 3, 4, 5, 6,
                                                             7, 8, 9],
                                        'min_samples_split': [2, 3, 4, 5, 6, 7,
                                                              8, 9, 10],
                                        'random_state': [30, 31, 32, 33, 34, 35,
                                                         36, 37, 38, 39, 40, 41,
                                                         42, 43, 44, 45, 46, 47,
                                                         48, 49, 50, 51, 52, 53,
                                                         54, 55, 56, 57, 58,
                                                         59]},
                   scoring='accuracy', verbose=1)

In [133]:
best_params = rs.best_params_
best_params

{'random_state': 36,
 'min_samples_split': 9,
 'min_samples_leaf': 8,
 'max_features': 'sqrt',
 'max_depth': 4,
 'criterion': 'log_loss'}

In [134]:
rs.best_score_

np.float64(0.575171204543701)

In [135]:
dtc = DecisionTreeClassifier(**best_params)

In [136]:
dtc.fit(x_train, y_train)

DecisionTreeClassifier(criterion='log_loss', max_depth=4, max_features='sqrt',
                       min_samples_leaf=8, min_samples_split=9,
                       random_state=36)

In [137]:
y_pred = dtc.predict(x_test)

In [138]:
print(accuracy_score(y_test, y_pred))
print(precision_score(y_test, y_pred))
print(recall_score(y_test, y_pred))
print(f1_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

0.5726605643392088
0.5638772475327836
0.6325447376402791
0.5962404402830391
[[3399 3226]
 [2423 4171]]


# Рандомный лес

In [141]:
rfc = RandomForestClassifier()

param_grid = {
    "bootstrap": [True, False],
    "max_depth": list(range(4, 100)),
    "max_features": ["sqrt", "log2"],
    "min_samples_leaf": list(range(1, 10)),
    "min_samples_split": list(range(2, 11)),
    "random_state": list(range(30, 60)),
    "n_estimators": list(range(50, 150))
}

random_search_rfc = RandomizedSearchCV(
    estimator=rfc,
    param_distributions=param_grid,
    n_iter=50,
    scoring="accuracy",
    verbose=1,
    n_jobs=-1
    )

random_search_rfc.fit(x_train, y_train["bind"])

Fitting 5 folds for each of 50 candidates, totalling 250 fits


RandomizedSearchCV(estimator=RandomForestClassifier(), n_iter=50, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [4, 5, 6, 7, 8, 9, 10, 11,
                                                      12, 13, 14, 15, 16, 17,
                                                      18, 19, 20, 21, 22, 23,
                                                      24, 25, 26, 27, 28, 29,
                                                      30, 31, 32, 33, ...],
                                        'max_features': ['sqrt', 'log2'],
                                        'min_samples_leaf': [1, 2, 3, 4, 5, 6,
                                                             7, 8, 9],
                                        'min_samples_split': [2, 3, 4, 5, 6, 7,
                                                              8, 9, 10],
                                        'n_estimators': [50, 51, 52, 53, 54, 55,
                                                         56, 57, 58, 59, 60, 61,
                                                         62, 63, 64, 65, 66, 67,
                                                         68, 69, 70, 71, 72, 73,
                                                         74, 75, 76, 77, 78, 79, ...],
                                        'random_state': [30, 31, 32, 33, 34, 35,
                                                         36, 37, 38, 39, 40, 41,
                                                         42, 43, 44, 45, 46, 47,
                                                         48, 49, 50, 51, 52, 53,
                                                         54, 55, 56, 57, 58,
                                                         59]},
                   scoring='accuracy', verbose=1)

In [142]:
rfc = RandomForestClassifier(**random_search_rfc.best_params_)

In [147]:
random_search_rfc.best_params_

{'random_state': 51,
 'n_estimators': 108,
 'min_samples_split': 4,
 'min_samples_leaf': 8,
 'max_features': 'sqrt',
 'max_depth': 84,
 'bootstrap': False}

In [144]:
rfc.fit(x_train, y_train["bind"])

RandomForestClassifier(bootstrap=False, max_depth=84, min_samples_leaf=8,
                       min_samples_split=4, n_estimators=108, random_state=51)

In [145]:
y_pred = rfc.predict(x_test)

In [146]:
print(accuracy_score(y_test, y_pred))
print(precision_score(y_test, y_pred))
print(recall_score(y_test, y_pred))
print(f1_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

0.7048944700809441
0.6893545211643932
0.7434030937215651
0.7153593578985772
[[4416 2209]
 [1692 4902]]
